# Course1-Project1: EdTech Backend System

In [19]:
import hashlib
from enum import Enum

class Role(Enum):
    LEARNER = "Learner"
    INSTRUCTOR = "Instructor"
        
class User:
    def __init__(self, name, role, email=None, password=None):
        '''
        'name' and 'role' are required at object creation.
        'role' can be only either 'learner' or 'instructor'
        'email' and 'password' are optional.
        '''
        if not isinstance(role,Role):
            raise ValueError("role must be a member of Role Enum.")
            
        self.name = name
        self.role = role
        self.email = email
        #self.__password = password #private attr.
        self.__password_hash = None

        if password:
            self.set_password(password)

    def set_email(self, email):
        self.email=email

    def validate_email(self,email):
        return self.email == email

    def set_password(self, password):
        '''
        Hash the password using SHA-256 before storing it.
        '''
        # encode to bytes and hash with sha256
        hashed = hashlib.sha256(password.encode("utf-8")).hexdigest()
        self.__password_hash = hashed

    def validate_password(self,password):
        return self.password == password

    def print_info(self):
        print("Name    :", self.name)
        print("Role    :", self.role.value)
        print("Email   :", self.email)
        print("Password:",self.__password)



u1 = User("Alice", Role.LEARNER,"alice@gmail.com","123")
u2 = User("Bob",Role.INSTRUCTOR)
u1.print_info()
u2.set_email("bob@gmail.com")
u2.print_info()
if (not u2.validate_email("bob2@example.com")):
    print("Wrong email")
#help(u1)


Name    : Alice
Role    : Learner
Email   : alice@gmail.com
Password: 123
Name    : Bob
Role    : Instructor
Email   : bob@gmail.com
Password: None
Wrong email
